In [2]:
import numpy as np
import pandas as pd
import seaborn as sb
import io
import requests
import datetime
import ondemand
from scipy.stats import pearsonr
from pandas.io.common import EmptyDataError
from requests.exceptions import HTTPError
import time
import os
import json
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [71]:
from multiprocessing import Pool as ThreadPool
from multiprocessing import Manager
class Generate_Leaders():
    def __init__(self): 
        self.pageview_change = dict()
        self.my_date = None
    def get_mkt_cap(self, symbol, start_date, end_date):
        file = pd.read_csv('Highest_change_total.csv')
        try:
            mktcap = float(file.loc[file['Symbol'] == symbol]['Market_Cap(billion)'])
        except:
            return
        return mktcap
    def parallel(self, item):
        try:
            pageview_price = pd.read_csv('pageview_price/pageview_price_{}.csv'.format(item.lower()))
        except FileNotFoundError:
            print('{} has no pageview data!!!'.format(item))
            return
        if pageview_price.empty:
            return
        #print(datetime.datetime.now())
        temp_res = pageview_price.set_index(['Date'])
        try:
            temp_res = temp_res.loc[[self.my_date]]
            temp_res.reset_index(col_fill = 'Date', inplace = True)
        except:
            print('{} has no trading on {}!!!'.format(item, self.my_date))
            return
        temp_res.loc[0, 'Market_Cap(billion)'] = self.get_mkt_cap(item, self.my_date, self.my_date)
        if temp_res['Market_Cap(billion)'].isnull().values.any():
            pass
        else:
            temp_res['Market_Cap(billion)'] = temp_res['Market_Cap(billion)'].round(2)
        self.pageview_change[item] = temp_res 
    def highest_change(self, my_date):
        df1 = pd.read_csv('NYSE_EQUITY_2019-01-14.csv')
        df2 = pd.read_csv('NASDAQ_2019-01-14.csv')
        symbol_NYSE = list(df1.symbol)
        symbol_NASDAQ = list(df2.symbol)
        symbols = symbol_NYSE + symbol_NASDAQ
        self.my_date = my_date
        for i, item in enumerate(symbols):
            if i % 300 == 0:
                print('run well')
                print(i)
            self.parallel(item)

In [59]:
df1 = pd.read_csv('NYSE_EQUITY_2019-01-14.csv')
df2 = pd.read_csv('NASDAQ_2019-01-14.csv')
symbol_NYSE = list(df1.symbol)
symbol_NASDAQ = list(df2.symbol)
all_symbols = symbol_NYSE + symbol_NASDAQ

In [67]:
daterange = []
start = '2018-06-14'
for i in range(17):
    daterange.append(start)
    temp = datetime.datetime.strptime(start, "%Y-%m-%d")
    temp += datetime.timedelta(days = 1)
    start = str(temp)[:10]

In [69]:
for i, date in enumerate(daterange):
    print(i)
    print(date)
    leader = Generate_Leaders()
    highest_change = leader.highest_change(date)
    to_combine = pd.DataFrame([])
    for item in all_symbols:
        try:
            temp = leader.pageview_change[item]
        except:
            continue
        to_combine = pd.concat([to_combine, temp], sort = False, ignore_index = True)
    if not os.path.exists(os.path.dirname('leaderlist {}/'.format(date))):
        os.makedirs(os.path.dirname('leaderlist {}/'.format(date)))
    if to_combine.empty:
        print('{} is not trading day!!!'.format(date))
        continue
    to_combine.to_csv('leaderlist {}/Highest_change_total.csv'.format(date), index = False)
    highest_change_window1 = to_combine.drop(['PageView_Change_in_Std_3', 'PageView_Change_in_Std_5', 'PageView_Change_in_Std_7', 'PageView_Change_in_Std_30'], axis = 1)
    highest_change_window1['Time_Window'] = 1
    highest_change_window1.sort_values(by = ['PageView_Change_in_Std_1'], ascending = False, inplace = True)
    highest_change_window3 = to_combine.drop(['PageView_Change_in_Std_1', 'PageView_Change_in_Std_5', 'PageView_Change_in_Std_7', 'PageView_Change_in_Std_30'], axis = 1)
    highest_change_window3['Time_Window'] = 3
    highest_change_window3.sort_values(by = ['PageView_Change_in_Std_3'], ascending = False, inplace = True)
    highest_change_window1.to_csv('leaderlist {}/highest_change_window1.csv'.format(date), index = False)
    highest_change_window3.to_csv('leaderlist {}/highest_change_window3.csv'.format(date), index = False)
    

0
2018-06-14
run well
0
ABR-B has no trading on 2018-06-14!!!
ACA has no trading on 2018-06-14!!!
AMH-H has no trading on 2018-06-14!!!
AQNA has no trading on 2018-06-14!!!
ARLO has no trading on 2018-06-14!!!
ASB-E has no trading on 2018-06-14!!!
AVLR has no trading on 2018-06-14!!!
BAC-K has no trading on 2018-06-14!!!
run well
300
BC-A has no trading on 2018-06-14!!!
BC-B has no pageview data!!!
BCSF has no trading on 2018-06-14!!!
BE has no trading on 2018-06-14!!!
BHR-D has no trading on 2018-06-14!!!
BJ has no trading on 2018-06-14!!!
BV has no trading on 2018-06-14!!!
CAI-B has no trading on 2018-06-14!!!
CCC has no trading on 2018-06-14!!!
CCC.U has no trading on 2018-06-14!!!
CCC.W has no trading on 2018-06-14!!!
CCH.U has no trading on 2018-06-14!!!
CHAP has no trading on 2018-06-14!!!
CIM-C has no trading on 2018-06-14!!!
CL-L has no pageview data!!!
run well
600
CMSC has no trading on 2018-06-14!!!
CNF has no trading on 2018-06-14!!!
CNP-B has no trading on 2018-06-14!!!
CP

GPAQW has no trading on 2018-06-14!!!
GRIN has no trading on 2018-06-14!!!
GRSHU has no trading on 2018-06-14!!!
GRTS has no trading on 2018-06-14!!!
HROW has no pageview data!!!
HUNTW has no trading on 2018-06-14!!!
HYAC has no trading on 2018-06-14!!!
HYRE has no trading on 2018-06-14!!!
HYXE has no trading on 2018-06-14!!!
IIIV has no trading on 2018-06-14!!!
IPLDP has no pageview data!!!
run well
4500
JASNW has no trading on 2018-06-14!!!
JG has no trading on 2018-06-14!!!
JSYNW has no trading on 2018-06-14!!!
KLXE has no trading on 2018-06-14!!!
KOD has no trading on 2018-06-14!!!
KZR has no trading on 2018-06-14!!!
LALT has no trading on 2018-06-14!!!
LDSF has no pageview data!!!
LEGH has no trading on 2018-06-14!!!
LFAC has no trading on 2018-06-14!!!
LOGC has no trading on 2018-06-14!!!
LOVE has no trading on 2018-06-14!!!
LQDA has no trading on 2018-06-14!!!
LSBK has no trading on 2018-06-14!!!
MDJH has no pageview data!!!
MESA has no trading on 2018-06-14!!!
MGTA has no tradi

NOM has no trading on 2018-06-15!!!
OAK-A has no trading on 2018-06-15!!!
OAK-B has no trading on 2018-06-15!!!
run well
2100
PBC has no trading on 2018-06-15!!!
PBY has no trading on 2018-06-15!!!
PEB-E has no pageview data!!!
PEB-F has no pageview data!!!
PLAN has no trading on 2018-06-15!!!
PRI-A has no trading on 2018-06-15!!!
PRI-B has no pageview data!!!
PRS has no trading on 2018-06-15!!!
PYT has no trading on 2018-06-15!!!
QTS-B has no trading on 2018-06-15!!!
QVCD has no trading on 2018-06-15!!!
REX-A has no trading on 2018-06-15!!!
REZI has no trading on 2018-06-15!!!
RMED has no trading on 2018-06-15!!!
RMI has no trading on 2018-06-15!!!
RVI has no trading on 2018-06-15!!!
SAF has no trading on 2018-06-15!!!
run well
2400
SDRL has no trading on 2018-06-15!!!
SIT-A has no pageview data!!!
SIT-J has no pageview data!!!
SIT-K has no pageview data!!!
SPAQ has no trading on 2018-06-15!!!
SRE-B has no trading on 2018-06-15!!!
SSW-I has no trading on 2018-06-15!!!
SWI has no tradi

SG has no trading on 2018-06-15!!!
SGLBW has no trading on 2018-06-15!!!
SHIPW has no trading on 2018-06-15!!!
SIBN has no trading on 2018-06-15!!!
SIC has no trading on 2018-06-15!!!
SONO has no trading on 2018-06-15!!!
STIM has no trading on 2018-06-15!!!
STNE has no trading on 2018-06-15!!!
STNL has no trading on 2018-06-15!!!
STNLU has no trading on 2018-06-15!!!
STRO has no trading on 2018-06-15!!!
SVMK has no trading on 2018-06-15!!!
SXTC has no pageview data!!!
TBIO has no trading on 2018-06-15!!!
run well
5700
TBLT has no trading on 2018-06-15!!!
TBRG has no trading on 2018-06-15!!!
TC has no trading on 2018-06-15!!!
TCDA has no trading on 2018-06-15!!!
TENB has no trading on 2018-06-15!!!
THOR has no trading on 2018-06-15!!!
TIGO has no pageview data!!!
TKKS has no trading on 2018-06-15!!!
TKKSU has no trading on 2018-06-15!!!
TKKSW has no trading on 2018-06-15!!!
TLC has no trading on 2018-06-15!!!
TLRY has no trading on 2018-06-15!!!
TMCX has no trading on 2018-06-15!!!
TMSR

ARE has no trading on 2018-06-16!!!
ARE-A has no trading on 2018-06-16!!!
ARES has no trading on 2018-06-16!!!
ARGD has no trading on 2018-06-16!!!
ARGO has no trading on 2018-06-16!!!
ARI has no trading on 2018-06-16!!!
ARI-C has no trading on 2018-06-16!!!
ARL has no trading on 2018-06-16!!!
ARLO has no trading on 2018-06-16!!!
ARMK has no trading on 2018-06-16!!!
ARNC has no trading on 2018-06-16!!!
AROC has no trading on 2018-06-16!!!
ARR has no trading on 2018-06-16!!!
ARR-A has no trading on 2018-06-16!!!
ARR-B has no trading on 2018-06-16!!!
ARW has no trading on 2018-06-16!!!
ASA has no trading on 2018-06-16!!!
ASB has no trading on 2018-06-16!!!
ASB-C has no trading on 2018-06-16!!!
ASB-D has no trading on 2018-06-16!!!
ASB-E has no trading on 2018-06-16!!!
ASC has no trading on 2018-06-16!!!
ASG has no trading on 2018-06-16!!!
ASGN has no trading on 2018-06-16!!!
ASH has no trading on 2018-06-16!!!
ASIX has no trading on 2018-06-16!!!
ASPN has no trading on 2018-06-16!!!
ASR 

...

PATK has no trading on 2018-06-30!!!
PAVM has no trading on 2018-06-30!!!
PAVMZ has no trading on 2018-06-30!!!
PAYX has no trading on 2018-06-30!!!
PBBI has no trading on 2018-06-30!!!
PBCT has no trading on 2018-06-30!!!
PBCTP has no trading on 2018-06-30!!!
PBIP has no trading on 2018-06-30!!!
PBPB has no trading on 2018-06-30!!!
PBYI has no trading on 2018-06-30!!!
PCAR has no trading on 2018-06-30!!!
PCB has no trading on 2018-06-30!!!
PCH has no trading on 2018-06-30!!!
PCMI has no trading on 2018-06-30!!!
PCOM has no trading on 2018-06-30!!!
PCRX has no trading on 2018-06-30!!!
PCSB has no trading on 2018-06-30!!!
PCTI has no trading on 2018-06-30!!!
PCTY has no trading on 2018-06-30!!!
PCYG has no trading on 2018-06-30!!!
PCYO has no trading on 2018-06-30!!!
PDBC has no trading on 2018-06-30!!!
PDCE has no trading on 2018-06-30!!!
PDCO has no trading on 2018-06-30!!!
PDD has no trading on 2018-06-30!!!
PDEX has no trading on 2018-06-30!!!
PDFS has no trading on 2018-06-30!!!
PD

RDVY has no trading on 2018-06-30!!!
RDWR has no trading on 2018-06-30!!!
RECN has no trading on 2018-06-30!!!
REDU has no trading on 2018-06-30!!!
REFR has no trading on 2018-06-30!!!
REG has no trading on 2018-06-30!!!
REGI has no trading on 2018-06-30!!!
REGN has no trading on 2018-06-30!!!
RELL has no trading on 2018-06-30!!!
RELV has no trading on 2018-06-30!!!
REPH has no trading on 2018-06-30!!!
REPL has no trading on 2018-06-30!!!
RESN has no trading on 2018-06-30!!!
RETA has no trading on 2018-06-30!!!
RETO has no trading on 2018-06-30!!!
RFAP has no trading on 2018-06-30!!!
RFDI has no trading on 2018-06-30!!!
RFEM has no trading on 2018-06-30!!!
RFEU has no trading on 2018-06-30!!!
RFIL has no trading on 2018-06-30!!!
RGCO has no trading on 2018-06-30!!!
RGEN has no trading on 2018-06-30!!!
RGLD has no trading on 2018-06-30!!!
RGLS has no trading on 2018-06-30!!!
RGNX has no trading on 2018-06-30!!!
RGSE has no trading on 2018-06-30!!!
RIBT has no trading on 2018-06-30!!!
RI

SNY has no trading on 2018-06-30!!!
SOCL has no trading on 2018-06-30!!!
SOHO has no trading on 2018-06-30!!!
SOHOB has no trading on 2018-06-30!!!
SOHOK has no trading on 2018-06-30!!!
SOHOO has no trading on 2018-06-30!!!
SOHU has no trading on 2018-06-30!!!
SOLO has no trading on 2018-06-30!!!
SONA has no trading on 2018-06-30!!!
SONO has no trading on 2018-06-30!!!
SORL has no trading on 2018-06-30!!!
SOXX has no trading on 2018-06-30!!!
SP has no trading on 2018-06-30!!!
SPAR has no trading on 2018-06-30!!!
SPCB has no trading on 2018-06-30!!!
SPEX has no trading on 2018-06-30!!!
SPHS has no trading on 2018-06-30!!!
SPI has no trading on 2018-06-30!!!
SPKE has no trading on 2018-06-30!!!
SPKEP has no trading on 2018-06-30!!!
SPLK has no trading on 2018-06-30!!!
SPNE has no trading on 2018-06-30!!!
SPNS has no trading on 2018-06-30!!!
SPOK has no trading on 2018-06-30!!!
SPPI has no trading on 2018-06-30!!!
SPRO has no trading on 2018-06-30!!!
SPRT has no trading on 2018-06-30!!!
S

TWIN has no trading on 2018-06-30!!!
TWLV has no trading on 2018-06-30!!!
TWLVR has no trading on 2018-06-30!!!
TWLVU has no trading on 2018-06-30!!!
TWLVW has no trading on 2018-06-30!!!
TWMC has no trading on 2018-06-30!!!
TWNK has no trading on 2018-06-30!!!
TWNKW has no trading on 2018-06-30!!!
TWOU has no trading on 2018-06-30!!!
TWST has no trading on 2018-06-30!!!
TXMD has no trading on 2018-06-30!!!
TXN has no trading on 2018-06-30!!!
TXRH has no trading on 2018-06-30!!!
TYHT has no trading on 2018-06-30!!!
TYME has no trading on 2018-06-30!!!
TYPE has no trading on 2018-06-30!!!
TZOO has no trading on 2018-06-30!!!
UAE has no trading on 2018-06-30!!!
UAL has no trading on 2018-06-30!!!
UBCP has no trading on 2018-06-30!!!
UBFO has no trading on 2018-06-30!!!
UBIO has no trading on 2018-06-30!!!
UBNK has no trading on 2018-06-30!!!
UBNT has no trading on 2018-06-30!!!
UBOH has no trading on 2018-06-30!!!
UBSH has no trading on 2018-06-30!!!
UBSI has no trading on 2018-06-30!!!


XNCR has no trading on 2018-06-30!!!
XNET has no trading on 2018-06-30!!!
XOG has no trading on 2018-06-30!!!
XOMA has no trading on 2018-06-30!!!
XON has no trading on 2018-06-30!!!
XONE has no trading on 2018-06-30!!!
XPER has no trading on 2018-06-30!!!
XRAY has no trading on 2018-06-30!!!
XSPA has no trading on 2018-06-30!!!
XT has no trading on 2018-06-30!!!
XTLB has no trading on 2018-06-30!!!
YGYI has no trading on 2018-06-30!!!
YI has no trading on 2018-06-30!!!
YIN has no trading on 2018-06-30!!!
YLCO has no trading on 2018-06-30!!!
YLDE has no trading on 2018-06-30!!!
YMAB has no trading on 2018-06-30!!!
YNDX has no trading on 2018-06-30!!!
YOGA has no trading on 2018-06-30!!!
YORW has no trading on 2018-06-30!!!
YRCW has no trading on 2018-06-30!!!
YRIV has no trading on 2018-06-30!!!
YTEN has no trading on 2018-06-30!!!
YTRA has no trading on 2018-06-30!!!
YVR has no trading on 2018-06-30!!!
YY has no trading on 2018-06-30!!!
Z has no trading on 2018-06-30!!!
ZAGG has no tr

In [3]:
daterange1 = []
start = '2018-05-31'
for i in range(31):
    daterange1.append(start)
    temp = datetime.datetime.strptime(start, "%Y-%m-%d")
    temp += datetime.timedelta(days = 1)
    start = str(temp)[:10]

In [16]:
#separate group for equities
for win in [1, 3]:
    for each in daterange1:
        try:
            highest_change_window1 = pd.read_csv('leaderlist {}/highest_change_window{}.csv'.format(each, win))
        except FileNotFoundError:
            continue
        small_cap_highest_change = highest_change_window1.loc[highest_change_window1['Market_Cap(billion)'] < 2]
        temp = highest_change_window1.loc[highest_change_window1['Market_Cap(billion)'] >= 2]
        mid_cap_highest_change = temp.loc[temp['Market_Cap(billion)'] < 10]
        large_cap_highest_change = temp.loc[temp['Market_Cap(billion)'] >= 10]
        if not os.path.exists(os.path.dirname('leaderlist {}/small_cap/window_{}_day/'.format(each, win))):
            os.makedirs(os.path.dirname('leaderlist {}/small_cap/window_{}_day/'.format(each, win)))
        if not os.path.exists(os.path.dirname('leaderlist {}/mid_cap/window_{}_day/'.format(each, win))):
            os.makedirs(os.path.dirname('leaderlist {}/mid_cap/window_{}_day/'.format(each, win)))
        if not os.path.exists(os.path.dirname('leaderlist {}/large_cap/window_{}_day/'.format(each, win))):
            os.makedirs(os.path.dirname('leaderlist {}/large_cap/window_{}_day/'.format(each, win)))
        small_cap_highest_change.to_csv('leaderlist {}/small_cap/window_{}_day/small_cap_highest_change_time_window_{}.csv'.format(each, win, win), index = False)
        mid_cap_highest_change.to_csv('leaderlist {}/mid_cap/window_{}_day/mid_cap_highest_change_time_window_{}.csv'.format(each, win, win), index = False)
        large_cap_highest_change.to_csv('leaderlist {}/large_cap/window_{}_day/large_cap_highest_change_time_window_{}.csv'.format(each, win, win), index = False)
    
    
    

here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here


In [18]:
#Pick the top 10 symbols with highest PageView_Change_in_std and PageView > 10
for win in [1, 3]:
    for date in daterange1:
        for cap in ['small', 'mid', 'large']:
            try:
                to_filter = pd.read_csv\
                ('leaderlist {}/{}_cap/window_{}_day/{}_cap_highest_change_time_window_{}.csv'.format(date, cap, win, cap, win))
            except FileNotFoundError:
                continue
            each = date
            each = each[5:]
            each = each.replace('-', '')
            to_filter = to_filter.loc[to_filter['PageView'] > 10]
            to_filter.reset_index(drop = True, inplace = True)
            to_filter = to_filter.iloc[0:10]
            to_filter.to_csv\
            ('leaderlist {}/{}_cap/window_{}_day/{}_{}_cap_window_{}_symbols.csv'.format(date, cap, win, each, cap, win), index = False)
            

In [25]:
#back test for 1-business day strategy
for win in [1, 3]:
    for date in daterange1:
        print(date)
        for cap in ['small', 'mid', 'large']:
            url = 'https://ondemand.websol.barchart.com/getHistory.csv?apikey=onDemand&symbol={}&startDate={}&endDate={}&type=daily&splits=true&nearby=1&jerq=true&exchange=NYSE%2CAMEX%2CNASDAQ'
            trading_day = datetime.datetime.strptime(date, "%Y-%m-%d")
            trading_day += datetime.timedelta(days = 1)
            while True:
                try:
                    temp = pd.read_csv(url.format('AAPL', trading_day, trading_day))
                    break
                except:
                    trading_day += datetime.timedelta(days = 1)
            trading_day = str(trading_day)[:10]
            each = date
            each = each[5:]
            each = each.replace('-', '')
            try:
                portfolio = pd.read_csv\
            ('leaderlist {}/{}_cap/window_{}_day/{}_{}_cap_window_{}_symbols.csv'.format(date, cap, win, each, cap, win))
            except FileNotFoundError:
                break
            portfolio = list(portfolio['Symbol'])
            invest = 0
            revenue = 0
            for item in portfolio:
                price = pd.read_csv(url.format(item, trading_day, trading_day))
                invest += price.loc[0, 'open']
                revenue += price.loc[0, 'close']
            trading_each = trading_day
            trading_each = trading_each[5:]
            trading_each = trading_each.replace('-', '')
            result = pd.DataFrame([], columns = ['Date', 'invest', 'hold(days)', 'portfolio_value', 'p&l', 'pct_change(%)', 'window'])
            result.loc[0, ['Date', 'invest', 'hold(days)', 'portfolio_value',\
                       'p&l', 'pct_change(%)', 'window']] = [trading_day, invest, 1, revenue, revenue-invest, (revenue/invest - 1)*100, win]
            if not os.path.exists(os.path.dirname('leaderlist {}/{}_cap/window_{}_day/hold_1_day/'.format(date, cap, win))):
                os.makedirs(os.path.dirname('leaderlist {}/{}_cap/window_{}_day/hold_1_day/'.format(date, cap, win)))
            result.to_csv('leaderlist {}/{}_cap/window_{}_day/hold_1_day/{}_{}_cap_window_{}_hold_1_pl.csv'.format(date, cap, win, trading_each, cap, win), index = False)
            

2018-05-31
2018-06-01
2018-06-02
2018-06-03
2018-06-04
2018-06-05
2018-06-06
2018-06-07
2018-06-08
2018-06-09
2018-06-10
2018-06-11
2018-06-12
2018-06-13
2018-06-14
2018-06-15
2018-06-16
2018-06-17
2018-06-18
2018-06-19
2018-06-20
2018-06-21
2018-06-22
2018-06-23
2018-06-24
2018-06-25
2018-06-26
2018-06-27
2018-06-28
2018-06-29
2018-06-30
2018-05-31
2018-06-01
2018-06-02
2018-06-03
2018-06-04
2018-06-05
2018-06-06
2018-06-07
2018-06-08
2018-06-09
2018-06-10
2018-06-11
2018-06-12
2018-06-13
2018-06-14
2018-06-15
2018-06-16
2018-06-17
2018-06-18
2018-06-19
2018-06-20
2018-06-21
2018-06-22
2018-06-23
2018-06-24
2018-06-25
2018-06-26
2018-06-27
2018-06-28
2018-06-29
2018-06-30


In [22]:
#Generate trading detail for 1-business day
for win in [1, 3]:
    for date in daterange1:
        print(date)
        for cap in ['small', 'mid', 'large']:
            url = 'https://ondemand.websol.barchart.com/getHistory.csv?apikey=onDemand&symbol={}&startDate={}&endDate={}&type=daily&splits=true&nearby=1&jerq=true&exchange=NYSE%2CAMEX%2CNASDAQ'
            trading_day = datetime.datetime.strptime(date, "%Y-%m-%d")
            trading_day += datetime.timedelta(days = 1)
            while True:
                try:
                    temp = pd.read_csv(url.format('AAPL', trading_day, trading_day))
                    break
                except:
                    trading_day += datetime.timedelta(days = 1)
            trading_day = str(trading_day)[:10]
            each = date
            each = each[5:]
            each = each.replace('-', '')
            try:
                portfolio = pd.read_csv\
            ('leaderlist {}/{}_cap/window_{}_day/{}_{}_cap_window_{}_symbols.csv'.format(date, cap, win, each, cap, win))
            except FileNotFoundError:
                break
            portfolio = list(portfolio['Symbol'])
            trading_each = trading_day
            trading_each = trading_each[5:]
            trading_each = trading_each.replace('-', '')
            result = pd.DataFrame([], columns = ['symbol', 'buy_date', 'sell_date', 'buy_price', 'sell_price', 'p&l', 'pct_change(%)', 'window'])
            result['symbol'] = portfolio
            result['buy_date'] = trading_day
            result['sell_date'] = trading_day
            for i, item in enumerate(portfolio):
                price = pd.read_csv(url.format(item, trading_day, trading_day))
                result.loc[i, 'buy_price'] = price.loc[0, 'open']
                result.loc[i, 'sell_price'] = price.loc[0, 'close']
                result.loc[i, 'p&l'] = price.loc[0, 'close'] - price.loc[0, 'open']
                result.loc[i, 'pct_change(%)'] = (price.loc[0, 'close']/price.loc[0, 'open'] - 1)*100
                result.loc[i, 'window'] = win
            result.to_csv('leaderlist {}/{}_cap/window_{}_day/hold_1_day/{}_{}_cap_window_{}_hold_1_detail.csv'.format(date, cap, win, trading_each, cap, win), index = False)
            
            
            

2018-05-31
2018-06-01
2018-06-02
2018-06-03
2018-06-04
2018-06-05
2018-06-06
2018-06-07
2018-06-08
2018-06-09
2018-06-10
2018-06-11
2018-06-12
2018-06-13
2018-06-14
2018-06-15
2018-06-16
2018-06-17
2018-06-18
2018-06-19
2018-06-20
2018-06-21
2018-06-22
2018-06-23
2018-06-24
2018-06-25
2018-06-26
2018-06-27
2018-06-28
2018-06-29
2018-06-30
2018-05-31
2018-06-01
2018-06-02
2018-06-03
2018-06-04
2018-06-05
2018-06-06
2018-06-07
2018-06-08
2018-06-09
2018-06-10
2018-06-11
2018-06-12
2018-06-13
2018-06-14
2018-06-15
2018-06-16
2018-06-17
2018-06-18
2018-06-19
2018-06-20
2018-06-21
2018-06-22
2018-06-23
2018-06-24
2018-06-25
2018-06-26
2018-06-27
2018-06-28
2018-06-29
2018-06-30


In [34]:
#to summary the 1-business day strategy results
for win in [1, 3]:
    for cap in ['small', 'mid', 'large']:
        to_combine = pd.DataFrame([])
        for date in daterange1:
            trading_day = datetime.datetime.strptime(date, "%Y-%m-%d")
            trading_day += datetime.timedelta(days = 1)
            while True:
                try:
                    temp = pd.read_csv(url.format('AAPL', trading_day, trading_day))
                    break
                except:
                    trading_day += datetime.timedelta(days = 1)
            trading_day = str(trading_day)[:10]
            each = trading_day
            each = each[5:]
            each = each.replace('-', '')
            try:
                temp = pd.read_csv('leaderlist {}/{}_cap/window_{}_day/hold_1_day/{}_{}_cap_window_{}_hold_1_pl.csv'.format(date, cap, win, each, cap, win))
            except FileNotFoundError:
                continue
            to_combine = pd.concat([to_combine, temp], sort = False, ignore_index = True)
        to_combine.loc['total_p&l', 'p&l'] = sum(to_combine.loc[to_combine['window'] == win]['p&l'])
        to_combine.loc['average_pct_change', 'pct_change(%)'] = to_combine['pct_change(%)'].mean()
        to_combine.loc['total_p&l', 'Date'] = 'Total_P&L'
        to_combine.loc['average_pct_change', 'Date'] = 'Average_Pct_Change'
        if not os.path.exists(os.path.dirname('{}_cap/window_{}_day/'.format(cap, win))):
            os.makedirs(os.path.dirname('{}_cap/window_{}_day/'.format(cap, win)))
        to_combine.to_csv('{}_cap/window_{}_day/{}_cap_window_{}_hold_1_pl_summary.csv'.format(cap, win, cap, win), index = False)    

In [31]:
#back test for 3-business day strategy
for win in [1, 3]:
    for date in daterange1:
        print(date)
        for cap in ['small', 'mid', 'large']:
            url = 'https://ondemand.websol.barchart.com/getHistory.csv?apikey=onDemand&symbol={}&startDate={}&endDate={}&type=daily&splits=true&nearby=1&jerq=true&exchange=NYSE%2CAMEX%2CNASDAQ'
            trading_day = datetime.datetime.strptime(date, "%Y-%m-%d")
            trading_day += datetime.timedelta(days = 1)
            while True:
                try:
                    temp = pd.read_csv(url.format('AAPL', trading_day, trading_day))
                    break
                except:
                    trading_day += datetime.timedelta(days = 1)
            trading_day = str(trading_day)[:10]
            each = date
            each = each[5:]
            each = each.replace('-', '')
            try:
                portfolio = pd.read_csv\
            ('leaderlist {}/{}_cap/window_{}_day/{}_{}_cap_window_{}_symbols.csv'.format(date, cap, win, each, cap, win))
            except FileNotFoundError:
                break
            portfolio = list(portfolio['Symbol'])
            invest = 0
            revenue = 0
            for item in portfolio:
                buy_price = pd.read_csv(url.format(item, trading_day, trading_day))
                invest += buy_price.loc[0, 'open']
                sell_day = datetime.datetime.strptime(trading_day, "%Y-%m-%d")
                for j in range(2):
                    sell_day += datetime.timedelta(days = 1)
                    while True:
                        try:
                            temp = pd.read_csv(url.format('AAPL', sell_day, sell_day))
                            break
                        except:
                            sell_day += datetime.timedelta(days = 1)
                sell_price = pd.read_csv(url.format(item, sell_day, sell_day))
                revenue += sell_price.loc[0, 'close']
            trading_each = trading_day
            trading_each = trading_each[5:]
            trading_each = trading_each.replace('-', '')
            result = pd.DataFrame([], columns = ['Date', 'invest', 'hold(days)', 'portfolio_value', 'p&l', 'pct_change(%)', 'window'])
            result.loc[0, ['Date', 'invest', 'hold(days)', 'portfolio_value',\
                       'p&l', 'pct_change(%)', 'window']] = [trading_day, invest, 3, revenue, revenue-invest, (revenue/invest - 1)*100, win]
            if not os.path.exists(os.path.dirname('leaderlist {}/{}_cap/window_{}_day/hold_3_day/'.format(date, cap, win))):
                os.makedirs(os.path.dirname('leaderlist {}/{}_cap/window_{}_day/hold_3_day/'.format(date, cap, win)))
            result.to_csv('leaderlist {}/{}_cap/window_{}_day/hold_3_day/{}_{}_cap_window_{}_hold_3_pl.csv'.format(date, cap, win, trading_each, cap, win), index = False)
        

2018-05-31
2018-06-01
2018-06-02
2018-06-03
2018-06-04
2018-06-05
2018-06-06
2018-06-07
2018-06-08
2018-06-09
2018-06-10
2018-06-11
2018-06-12
2018-06-13
2018-06-14
2018-06-15
2018-06-16
2018-06-17
2018-06-18
2018-06-19
2018-06-20
2018-06-21
2018-06-22
2018-06-23
2018-06-24
2018-06-25
2018-06-26
2018-06-27
2018-06-28
2018-06-29
2018-06-30
2018-05-31
2018-06-01
2018-06-02
2018-06-03
2018-06-04
2018-06-05
2018-06-06
2018-06-07
2018-06-08
2018-06-09
2018-06-10
2018-06-11
2018-06-12
2018-06-13
2018-06-14
2018-06-15
2018-06-16
2018-06-17
2018-06-18
2018-06-19
2018-06-20
2018-06-21
2018-06-22
2018-06-23
2018-06-24
2018-06-25
2018-06-26
2018-06-27
2018-06-28
2018-06-29
2018-06-30


In [32]:
#Generate trading detail for 3-business day
for win in [1, 3]:
    for date in daterange1:
        print(date)
        for cap in ['small', 'mid', 'large']:
            url = 'https://ondemand.websol.barchart.com/getHistory.csv?apikey=onDemand&symbol={}&startDate={}&endDate={}&type=daily&splits=true&nearby=1&jerq=true&exchange=NYSE%2CAMEX%2CNASDAQ'
            trading_day = datetime.datetime.strptime(date, "%Y-%m-%d")
            trading_day += datetime.timedelta(days = 1)
            while True:
                try:
                    temp = pd.read_csv(url.format('AAPL', trading_day, trading_day))
                    break
                except:
                    trading_day += datetime.timedelta(days = 1)
            trading_day = str(trading_day)[:10]
            each = date
            each = each[5:]
            each = each.replace('-', '')
            try:
                portfolio = pd.read_csv\
            ('leaderlist {}/{}_cap/window_{}_day/{}_{}_cap_window_{}_symbols.csv'.format(date, cap, win, each, cap, win))
            except FileNotFoundError:
                break
            portfolio = list(portfolio['Symbol'])
            trading_each = trading_day
            trading_each = trading_each[5:]
            trading_each = trading_each.replace('-', '')
            sell_day = datetime.datetime.strptime(trading_day, "%Y-%m-%d")
            for j in range(2):
                sell_day += datetime.timedelta(days = 1)
                while True:
                    try:
                        temp = pd.read_csv(url.format('AAPL', sell_day, sell_day))
                        break
                    except:
                        sell_day += datetime.timedelta(days = 1)
            sell_price = pd.read_csv(url.format(item, sell_day, sell_day))
            result = pd.DataFrame([], columns = ['symbol', 'buy_date', 'sell_date', 'buy_price', 'sell_price', 'p&l', 'pct_change(%)', 'window'])
            result['symbol'] = portfolio
            result['buy_date'] = trading_day
            result['sell_date'] = sell_day
            for i, item in enumerate(portfolio):
                price = pd.read_csv(url.format(item, trading_day, sell_day))
                result.loc[i, 'buy_price'] = price.loc[0, 'open']
                result.loc[i, 'sell_price'] = price.loc[2, 'close']
                result.loc[i, 'p&l'] = price.loc[2, 'close'] - price.loc[0, 'open']
                result.loc[i, 'pct_change(%)'] = (price.loc[2, 'close']/price.loc[0, 'open'] - 1)*100
                result.loc[i, 'window'] = win
            result.to_csv('leaderlist {}/{}_cap/window_{}_day/hold_3_day/{}_{}_cap_window_{}_hold_3_detail.csv'.format(date, cap, win, trading_each, cap, win), index = False)
            
            
            

2018-05-31
2018-06-01
2018-06-02
2018-06-03
2018-06-04
2018-06-05
2018-06-06
2018-06-07
2018-06-08
2018-06-09
2018-06-10
2018-06-11
2018-06-12
2018-06-13
2018-06-14
2018-06-15
2018-06-16
2018-06-17
2018-06-18
2018-06-19
2018-06-20
2018-06-21
2018-06-22
2018-06-23
2018-06-24
2018-06-25
2018-06-26
2018-06-27
2018-06-28
2018-06-29
2018-06-30
2018-05-31
2018-06-01
2018-06-02
2018-06-03
2018-06-04
2018-06-05
2018-06-06
2018-06-07
2018-06-08
2018-06-09
2018-06-10
2018-06-11
2018-06-12
2018-06-13
2018-06-14
2018-06-15
2018-06-16
2018-06-17
2018-06-18
2018-06-19
2018-06-20
2018-06-21
2018-06-22
2018-06-23
2018-06-24
2018-06-25
2018-06-26
2018-06-27
2018-06-28
2018-06-29
2018-06-30


In [35]:
#to summary the 3-business day strategy results
for win in [1, 3]:
    for cap in ['small', 'mid', 'large']:
        to_combine = pd.DataFrame([])
        for date in daterange1:
            trading_day = datetime.datetime.strptime(date, "%Y-%m-%d")
            trading_day += datetime.timedelta(days = 1)
            while True:
                try:
                    temp = pd.read_csv(url.format('AAPL', trading_day, trading_day))
                    break
                except:
                    trading_day += datetime.timedelta(days = 1)
            trading_day = str(trading_day)[:10]
            each = trading_day
            each = each[5:]
            each = each.replace('-', '')
            try:
                temp = pd.read_csv('leaderlist {}/{}_cap/window_{}_day/hold_3_day/{}_{}_cap_window_{}_hold_3_pl.csv'.format(date, cap, win, each, cap, win))
            except FileNotFoundError:
                continue
            to_combine = pd.concat([to_combine, temp], sort = False, ignore_index = True)   
        to_combine.loc['total_p&l', 'p&l'] = sum(to_combine.loc[to_combine['window'] == win]['p&l'])
        to_combine.loc['average_pct_change', 'pct_change(%)'] = to_combine['pct_change(%)'].mean()
        to_combine.loc['total_p&l', 'Date'] = 'Total_P&L'
        to_combine.loc['average_pct_change', 'Date'] = 'Average_Pct_Change'
        if not os.path.exists(os.path.dirname('{}_cap/window_{}_day/'.format(cap, win))):
            os.makedirs(os.path.dirname('{}_cap/window_{}_day/'.format(cap, win)))
        to_combine.to_csv('{}_cap/window_{}_day/{}_cap_window_{}_hold_3_pl_summary.csv'.format(cap, win, cap, win), index = False)         

In [10]:
#back test for buy close, hold one day, and sell close price tomorrow
for win in [1, 3]:
    for date in daterange1:
        print(date)
        for cap in ['small', 'mid', 'large']:
            url = 'https://ondemand.websol.barchart.com/getHistory.csv?apikey=onDemand&symbol={}&startDate={}&endDate={}&type=daily&splits=true&nearby=1&jerq=true&exchange=NYSE%2CAMEX%2CNASDAQ'
            trading_day = date
            each = date
            each = each[5:]
            each = each.replace('-', '')
            try:
                portfolio = pd.read_csv\
            ('leaderlist {}/{}_cap/window_{}_day/{}_{}_cap_window_{}_symbols.csv'.format(date, cap, win, each, cap, win))
            except FileNotFoundError:
                break
            portfolio = list(portfolio['Symbol'])
            invest = 0
            portfolio_value = 0
            for item in portfolio:
                buy_price = pd.read_csv(url.format(item, trading_day, trading_day))
                invest += buy_price.loc[0, 'close']
                sell_day = datetime.datetime.strptime(trading_day, "%Y-%m-%d")
                sell_day += datetime.timedelta(days = 1)
                while True:
                    try:
                        temp = pd.read_csv(url.format('AAPL', sell_day, sell_day))
                        break
                    except:
                        sell_day += datetime.timedelta(days = 1)
                sell_price = pd.read_csv(url.format(item, sell_day, sell_day))
                portfolio_value += sell_price.loc[0, 'close']
            trading_each = trading_day
            trading_each = trading_each[5:]
            trading_each = trading_each.replace('-', '')
            result = pd.DataFrame([], columns = ['Date', 'invest', 'hold(days)', 'portfolio_value', 'p&l', 'pct_change(%)', 'window'])
            result.loc[0, ['Date', 'invest', 'hold(days)', 'portfolio_value',\
                       'p&l', 'pct_change(%)', 'window']] = [trading_day, invest, 1, portfolio_value, portfolio_value-invest, (portfolio_value/invest - 1)*100, win]
            if not os.path.exists(os.path.dirname('leaderlist {}/{}_cap/window_{}_day/hold_1b_day/'.format(date, cap, win))):
                os.makedirs(os.path.dirname('leaderlist {}/{}_cap/window_{}_day/hold_1b_day/'.format(date, cap, win)))
            result.to_csv('leaderlist {}/{}_cap/window_{}_day/hold_1b_day/{}_{}_cap_window_{}_hold_1b_pl.csv'.format(date, cap, win, trading_each, cap, win), index = False)


2018-05-31
2018-06-01
2018-06-02
2018-06-03
2018-06-04
2018-06-05
2018-06-06
2018-06-07
2018-06-08
2018-06-09
2018-06-10
2018-06-11
2018-06-12
2018-06-13
2018-06-14
2018-06-15
2018-06-16
2018-06-17
2018-06-18
2018-06-19
2018-06-20
2018-06-21
2018-06-22
2018-06-23
2018-06-24
2018-06-25
2018-06-26
2018-06-27
2018-06-28
2018-06-29
2018-06-30
2018-05-31
2018-06-01
2018-06-02
2018-06-03
2018-06-04
2018-06-05
2018-06-06
2018-06-07
2018-06-08
2018-06-09
2018-06-10
2018-06-11
2018-06-12
2018-06-13
2018-06-14
2018-06-15
2018-06-16
2018-06-17
2018-06-18
2018-06-19
2018-06-20
2018-06-21
2018-06-22
2018-06-23
2018-06-24
2018-06-25
2018-06-26
2018-06-27
2018-06-28
2018-06-29
2018-06-30


In [11]:
#Generate trading detail for today close to next day close
for win in [1, 3]:
    for date in daterange1:
        print(date)
        for cap in ['small', 'mid', 'large']:
            url = 'https://ondemand.websol.barchart.com/getHistory.csv?apikey=onDemand&symbol={}&startDate={}&endDate={}&type=daily&splits=true&nearby=1&jerq=true&exchange=NYSE%2CAMEX%2CNASDAQ'
            trading_day = date
            each = date
            each = each[5:]
            each = each.replace('-', '')
            try:
                portfolio = pd.read_csv\
            ('leaderlist {}/{}_cap/window_{}_day/{}_{}_cap_window_{}_symbols.csv'.format(date, cap, win, each, cap, win))
            except FileNotFoundError:
                break
            portfolio = list(portfolio['Symbol'])
            trading_each = trading_day
            trading_each = trading_each[5:]
            trading_each = trading_each.replace('-', '')
            sell_day = datetime.datetime.strptime(trading_day, "%Y-%m-%d")
            sell_day += datetime.timedelta(days = 1)
            while True:
                try:
                    temp = pd.read_csv(url.format('AAPL', sell_day, sell_day))
                    break
                except:
                    sell_day += datetime.timedelta(days = 1)
            result = pd.DataFrame([], columns = ['symbol', 'buy_date', 'sell_date', 'buy_price', 'sell_price', 'p&l', 'pct_change(%)', 'window'])
            result['symbol'] = portfolio
            result['buy_date'] = trading_day
            result['sell_date'] = sell_day
            for i, item in enumerate(portfolio):
                price = pd.read_csv(url.format(item, trading_day, sell_day))
                result.loc[i, 'buy_price'] = price.loc[0, 'close']
                result.loc[i, 'sell_price'] = price.loc[1, 'close']
                result.loc[i, 'p&l'] = price.loc[1, 'close'] - price.loc[0, 'close']
                result.loc[i, 'pct_change(%)'] = (price.loc[1, 'close']/price.loc[0, 'close'] - 1)*100
                result.loc[i, 'window'] = win
            result.to_csv('leaderlist {}/{}_cap/window_{}_day/hold_1b_day/{}_{}_cap_window_{}_hold_1b_detail.csv'.format(date, cap, win, trading_each, cap, win), index = False)
            
            

2018-05-31
2018-06-01
2018-06-02
2018-06-03
2018-06-04
2018-06-05
2018-06-06
2018-06-07
2018-06-08
2018-06-09
2018-06-10
2018-06-11
2018-06-12
2018-06-13
2018-06-14
2018-06-15
2018-06-16
2018-06-17
2018-06-18
2018-06-19
2018-06-20
2018-06-21
2018-06-22
2018-06-23
2018-06-24
2018-06-25
2018-06-26
2018-06-27
2018-06-28
2018-06-29
2018-06-30
2018-05-31
2018-06-01
2018-06-02
2018-06-03
2018-06-04
2018-06-05
2018-06-06
2018-06-07
2018-06-08
2018-06-09
2018-06-10
2018-06-11
2018-06-12
2018-06-13
2018-06-14
2018-06-15
2018-06-16
2018-06-17
2018-06-18
2018-06-19
2018-06-20
2018-06-21
2018-06-22
2018-06-23
2018-06-24
2018-06-25
2018-06-26
2018-06-27
2018-06-28
2018-06-29
2018-06-30


In [12]:
#to summary the today's close to next close strategy results
for win in [1, 3]:
    for cap in ['small', 'mid', 'large']:
        to_combine = pd.DataFrame([])
        for date in daterange1:
            trading_day = date
            each = trading_day
            each = each[5:]
            each = each.replace('-', '')
            try:
                temp = pd.read_csv('leaderlist {}/{}_cap/window_{}_day/hold_1b_day/{}_{}_cap_window_{}_hold_1b_pl.csv'.format(date, cap, win, each, cap, win))
            except FileNotFoundError:
                continue
            to_combine = pd.concat([to_combine, temp], sort = False, ignore_index = True)   
        to_combine.loc['total_p&l', 'p&l'] = sum(to_combine.loc[to_combine['window'] == win]['p&l'])
        to_combine.loc['average_pct_change', 'pct_change(%)'] = to_combine['pct_change(%)'].mean()
        to_combine.loc['total_p&l', 'Date'] = 'Total_P&L'
        to_combine.loc['average_pct_change', 'Date'] = 'Average_Pct_Change'
        if not os.path.exists(os.path.dirname('{}_cap/window_{}_day/'.format(cap, win))):
            os.makedirs(os.path.dirname('{}_cap/window_{}_day/'.format(cap, win)))
        to_combine.to_csv('{}_cap/window_{}_day/{}_cap_window_{}_hold_1b_pl_summary.csv'.format(cap, win, cap, win), index = False)
        
        